# Project: Yelp WAX

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The api, however, provides you with a lot of information that will not be pertinent to your analysis. YOu will pull data from the api and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the api will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

___

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

WAX:
https://www.yelp.com/developers/documentation/v3/business_search
ETL - extract, transform, load

In [1]:
import requests
import json
import csv
import os
from helper_funcs import *


In [2]:
# what type of business do you want to search
term = 'gym'

#where do you want to perform this search
location = 'Brooklyn'

# what is your other parameter you want to search against
categories = 'gyms'

In [3]:
url =  'https://api.yelp.com/v3/businesses/search'

In [4]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
            }

In [5]:
gyms = yelp_call(url_params, api_key)

In [6]:
gyms.keys();

In [7]:
gyms['businesses'];

In [8]:
gyms['businesses'][0];

In [9]:
gyms['total'];

In [10]:
gyms['region'];

In [11]:
gyms_parsed = parse_results(gyms['businesses'])

In [12]:
gyms_parsed_ld = parse_results_ld(gyms['businesses'])

In [13]:
gyms_parsed_ld;

In [14]:
len(gyms_parsed);

In [15]:
len(gyms_parsed_ld);

In [16]:
gyms_parsed_ld[0];

In [17]:
import os
cwd = os.getcwd()
cwd

'/Users/wax/Documents/Classes/FS Data Science Immersive/Flatiron_Materials/Feb 1/wax_yelp_project/code'

In [19]:
gyms = yelp_call(url_params, api_key)
gyms_parsed_ld = parse_results_ld(gyms['businesses'])
csv_append('../data/data.csv', gyms_parsed_ld)

- Write a script that combines the three functions above into a single process.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [22]:
# create a variable  to keep track of which result you are in. 
cur = 0
# max is 1000 businesses, total

# delete data.csv if it exists
csv_filepath = '../data/data.csv'
if os.path.exists(csv_filepath):
    os.remove(csv_filepath)
    print('Deleted existing data.csv file.')

#set up a while loop to go through and grab the result 
while cur < 1000:
# while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    #after you get your results you can now use your function to parse those results
    parsed_results_ld = parse_results_ld(results['businesses'])
    if cur == 0:
        data_fields = csv_create(csv_filepath, parsed_results_ld)
        print('Created new data.csv file and added headers:')
        print(list(data_fields))
        print('Downloading data -', end =" ")
    # use your function to insert your parsed results into the db
    csv_append(csv_filepath, parsed_results_ld)
    print('-', end =" ")
    #increment the counter by 50 to move on to the next results
    cur += 50
assert(cur==1000)
print('Done.'')
print(f'Successfully gathered listings for 1000 businesses of type \'{categories}\', search term \'{term}\', in \'{location}\'.')

Deleted existing data.csv file.
Created new data.csv file and added headers:
['id', 'name', 'is_closed', 'review_count', 'zip_code', 'rating']
Successfully gathered 1000 businesses.


___

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you ahve a model of the functions you will need to write, and how to pull them together in one script. For this part, you ahve the process below 

- In order to pull the reviews, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

In [ ]:
with open(csv_filepath, 'r') as csvfile:
    reader = csv.DictReader(csvfile, newline = '')
    reader.

- Write a function that takes a business id and makes a call to the API for reivews


- Write a function to parse out the relevant information from the reviews

- Write a function to save the parse data into a csv file containing all of the reviews. 

- Combine the functions above into a single script  

___

## Part 4 -  Using python and pandas, write code to answer the questions below. 


- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20